FFN output distribution for input gaussian-distributed coefficients

In [1]:
import math
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn

***
In the following example a FFN with Relu activation is studying. Input data is an array ${xx}$ of size ${n_0}$ x ${n_d}$, ${n_0}$ - input data dimension and ${n_d}$ - number of points in trainset. A series of ${experiments\_number}$ experiments is performed, each one generating FFN weights from a distribution at criticality. Output data is an array ${yy}$ of size ${experiments\_number}$ x ${n_l}$ x ${n_d}$, where ${n_l}$ - output data dimension.
***

In [2]:
def sample_covariance(neuron_one, trainpoint_one, neuron_two, trainpoint_two, yy):
    '''Sample covariance, formulas as in https://en.wikipedia.org/wiki/Sample_mean_and_covariance'''
    one = yy[:, neuron_one-1, trainpoint_one-1]
    two = yy[:, neuron_two-1, trainpoint_two-1]
    nn, mean_one, mean_two, sum = len(one), np.mean(one), np.mean(two), 0.0
    for pos in range(nn):
        sum += (one[pos]-mean_one)*(two[pos]-mean_two)

    return sum/(nn - 1)

def K_xx(trainpoint_one, trainpoint_two, cw, xx):
    '''Metric (4.8) calculation'''
    one, two = xx[:,trainpoint_one-1],xx[:,trainpoint_two-1]
    nn, sum = len(one), 0.0
    for pos in range(nn):
        sum += one[pos]*two[pos]

    return sum*cw/nn

In [3]:
class FeedForwardNet(nn.Module):

    def __init__(self, n0=3, nk=10, nl=3, l=3, bias_on=False):
        '''n0: # dimension of x
           nk: # hidden nodes
           nl: # dimension of y
           l: # number of layers
           bias_on: # whether bias is included into linear preactivations'''
        super().__init__()
        self.n0=n0
        self.nk=nk
        self.nl=nl
        self.bias_on = bias_on
        self.log_level = None
        self.linears = []
        print("FeedForwardNet created with n0={}, nk={}, nl={}, l={}, bias_on={}".format(n0, nk, nl, l, bias_on))

        self.linears.append(nn.Linear(n0, nk, bias=bias_on))
        if l > 2:
            for _ in range(2, l):
                self.linears.append(nn.Linear(nk, nk, bias=bias_on))
        if l > 1:
            self.linears.append(nn.Linear(nk, nl, bias=bias_on))

    def set_log_level(self, value):
        self.log_level = value

    def get_log_level(self):
        if self.log_level in ("debug", "info", "warning", "error"):
            return self.log_level
        else:
            return "info"

    def init_weights(self, cb=1.0, cw=1.0):
        if self.get_log_level() == "debug":
            print("FeedForwardNet weights initialised with cb={}, cw={}".format(cb, cw))

        #Weight initialisation as in 2.19, 2.20
        n_prev = self.n0
        for linear in self.linears:
            nn.init.normal_(linear.weight, mean = 0., std = math.sqrt(cw/n_prev))
            n_prev = linear.weight.size()[0]
            if self.bias_on:
                nn.init.normal_(linear.bias, mean = 0., std = math.sqrt(cb))



FFN with PReLU(slope_positive, slope_negative); 

In [4]:
class ParametricReLUNet(FeedForwardNet):
    def __init__(self, n0=3, nk=10, nl=3, l=3, bias_on=False):
        super().__init__(n0, nk, nl, l, bias_on)
        self.slope_positive = None
        self.slope_negative = None

    def set_slopes(self, slope_positive = 1.0, slope_negative = 0.25):
        self.slope_positive = slope_positive
        self.slope_negative = slope_negative

    def PReLU(self, input: Tensor) -> Tensor:
        for pos in range(input.size(dim=0)):
            input[pos] = input[pos] * (self.slope_positive if input[pos] >= 0 else self.slope_negative)
        return input

    def forward(self, xx):
        if self.slope_positive == None:
            raise Exception("To use forward set slopes with call ParametricReLUNet.set_slopes(...)")

        zk = torch.tensor(xx.transpose(), dtype=torch.float32)
        for linear in self.linears:
            zk = linear(zk)
            zk = self.PReLU(zk)
        return zk.detach().numpy().transpose()

In [6]:
#Test for PReLU-activation implementation

testPReLU = ParametricReLUNet()
testPReLU.set_slopes(0.5, 0.2)
resultPReLU = testPReLU.PReLU(torch.tensor(np.array([1.1,-2.2]), dtype=torch.float32))
print(resultPReLU)


FeedForwardNet created with n0=3, nk=10, nl=3, l=3, bias_on=False
tensor([ 0.5500, -0.4400])


In [13]:
'''n0: # dimension of x
    nk: # hidden nodes
    nl: # dimension of y
    l: # number of layers
    nd: # number of points in train-set'''
n0,nk,nl,l=3,10000,2,10
nd = 2
'''slope_plus, slope_minus: # slopes for Relu
    experiments_number: # number of experiments'''
slope_plus, slope_minus=1.0, 0.5
experiments_number = 20

testNet = ParametricReLUNet(n0=n0,nk=nk,nl=nl,l=l)
testNet.set_log_level("info")
testNet.set_slopes(slope_plus, slope_minus)
xx = np.random.normal(size=(n0, nd)).astype(np.float32)
yy = np.zeros((experiments_number, nl, nd))
cw= 2.0/(slope_plus**2.0 + slope_minus**2.0)

#for each experiment re-initialisation of the weights with recalculation
for experiment_number in range(experiments_number):
    #weights distribution is initialisied as in (5.67)
    testNet.init_weights(0, cw)
    for col in range(nd):
        res = testNet.forward(xx[:,col])
        for row in range(nl):
            yy[experiment_number,row,col] = res[row]

#print("xx:", xx)
#print("yy:", yy)

FeedForwardNet created with n0=3, nk=10000, nl=2, l=10, bias_on=False


In [14]:
for neuron1 in range(1, nl+1):
      for neuron2 in range(neuron1, nl+1):
            for trainpoint1 in range(1, nd+1):
                  for trainpoint2 in range(1 if neuron1 != neuron2 else trainpoint1, nd+1):
                        print("Sample covariance between neuron {}, trainpoint {} and neuron {}, trainpoint {}: {}"\
                              .format(neuron1, trainpoint1, neuron2, trainpoint2\
                                      , sample_covariance(neuron1,trainpoint1,neuron2,trainpoint2,yy)))


Sample covariance between neuron 1, trainpoint 1 and neuron 1, trainpoint 1: 0.9736863167494237
Sample covariance between neuron 1, trainpoint 1 and neuron 1, trainpoint 2: 1.758835041209648
Sample covariance between neuron 1, trainpoint 2 and neuron 1, trainpoint 2: 6.27316877308968
Sample covariance between neuron 1, trainpoint 1 and neuron 2, trainpoint 1: 0.012130154209039306
Sample covariance between neuron 1, trainpoint 1 and neuron 2, trainpoint 2: -0.3507023977958151
Sample covariance between neuron 1, trainpoint 2 and neuron 2, trainpoint 1: -0.11084808727733036
Sample covariance between neuron 1, trainpoint 2 and neuron 2, trainpoint 2: -0.624781140660716
Sample covariance between neuron 2, trainpoint 1 and neuron 2, trainpoint 1: 0.4404177515623559
Sample covariance between neuron 2, trainpoint 1 and neuron 2, trainpoint 2: -0.100767094206183
Sample covariance between neuron 2, trainpoint 2 and neuron 2, trainpoint 2: 6.4418267750269615


In [15]:
for trainpoint1 in range(1, nd+1):
    for trainpoint2 in range(trainpoint1, nd+1):
        print("Metric (4.8) for trainpoint {} and trainpoint {}: {}"\
              .format(trainpoint1, trainpoint2, K_xx(trainpoint1, trainpoint2, cw, xx)))


Metric (4.8) for trainpoint 1 and trainpoint 1: 1.1942569692929588
Metric (4.8) for trainpoint 1 and trainpoint 2: -0.5999009450276693
Metric (4.8) for trainpoint 2 and trainpoint 2: 6.032336298624674
